In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

while 'notebooks' in os.getcwd():
    os.chdir('..')

import pandas as pd
import torch
import torch.nn.functional as F
from ogb.nodeproppred import DglNodePropPredDataset, Evaluator
from sklearn.metrics import roc_auc_score
import logging
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import numpy as np

from src.dgl_models import GraphSAGE
from src.data.node_classifier.arxiv import load_dataset, get_symmetric_graph
from src.train.node_classifier import WeigthedGraphSageNodeClassifier

In [3]:
logging.basicConfig(
    format='%(asctime)s - %(levelname)s : %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [4]:
torch.cuda.is_available()

True

In [5]:
device = f'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

## Data Loading

In [6]:
dataset = load_dataset()

In [7]:
graph = get_symmetric_graph(dataset)

In [8]:
graph.number_of_edges()

2332486

In [9]:
torch.mean(graph.out_degrees() * 1.)

tensor(13.7737)

In [10]:
torch.mean(graph.in_degrees() * 1.)

tensor(13.7737)

In [11]:
split_idx = dataset.get_idx_split()

In [12]:
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

In [13]:
edges_gamma_df = pd.read_csv('data/gamma_graph_sage/01-1_gamma_scored_edges.csv')
print(edges_gamma_df.shape)
edges_gamma_df.head()

(2315598, 3)


,source,target,gamma_pred
0,411,0,0.649669
1,640,0,0.784250
2,1162,0,0.787807
3,1897,0,0.787201
4,3396,0,0.784839


In [14]:
keys = tuple(zip(edges_gamma_df['source'].values, edges_gamma_df['target'].values))
pred_dict = dict(zip(keys, edges_gamma_df['gamma_pred'].values))

In [15]:
idx=0
pred_dict[(graph.edges()[0][idx].item(), graph.edges()[1][idx].item())]

0.74366045

In [16]:
source = graph.edges()[0].numpy()
target = graph.edges()[1].numpy()
graph_edge_keys = tuple(zip(source, target))

In [17]:
edge_weights = [pred_dict[edge] for edge in graph_edge_keys]
edge_weights = torch.tensor(edge_weights, dtype=torch.float32).to(device)
edge_weights[:5]

tensor([0.7437, 0.7242, 0.5969, 0.7571, 0.7405], device='cuda:0')

In [18]:
features = graph.ndata['feat'].cuda()
labels = dataset.labels.cuda()
train_mask = split_idx['train'].cuda()
val_mask = split_idx['valid'].cuda()
test_mask = split_idx['test'].cuda()

In [19]:
train_nid = train_mask.nonzero().squeeze()
val_nid = val_mask.nonzero().squeeze()
test_nid = test_mask.nonzero().squeeze()

In [20]:
n_edges = graph.number_of_edges()

In [21]:
graph

Graph(num_nodes=169343, num_edges=2332486,
      ndata_schemes={'year': Scheme(shape=(1,), dtype=torch.int64), 'feat': Scheme(shape=(128,), dtype=torch.float32)}
      edata_schemes={})

In [22]:
torch.version.cuda

'11.3'

In [23]:
graph = graph.to(device)

In [24]:
n_layers = 3
n_iters = 3000
epochs = 5000
log_steps = 100
input_dim = features.shape[1]
hidden_channels = input_dim * 2
output_dim = dataset.num_classes
lr_rate = 0.001
dropout = 0.5

In [25]:
model = GraphSAGE(
    n_layers=n_layers,
    in_channels=input_dim,
    hidden_channels=hidden_channels,
    out_channels=output_dim,
    dropout=dropout)\
    .to(device)

In [26]:
def train(model, graph, features, train_mask, optimizer, edge_weight=None):
    model.train()

    optimizer.zero_grad()
    out = model(graph, features, edge_weight=edge_weight)[train_mask]
    loss = F.nll_loss(out, labels.squeeze(1)[train_mask])
    loss.backward()
    optimizer.step()

    return loss.item()

In [27]:
@torch.no_grad()
def test(model, graph, features, labels, train_mask, val_mask, test_mask, evaluator, edge_weight=None):
    model.eval()

    out = model(graph, features, edge_weight=edge_weight)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': labels[train_mask],
        'y_pred': y_pred[train_mask],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': labels[val_mask],
        'y_pred': y_pred[val_mask],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': labels[test_mask],
        'y_pred': y_pred[test_mask],
    })['acc']

    return train_acc, valid_acc, test_acc

In [28]:
evaluator = Evaluator(name='ogbn-arxiv')

In [29]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
for epoch in range(1, 1 + epochs):
    loss = train(model, graph, features, train_mask, optimizer, edge_weight=edge_weights)
    result = test(model, graph, features, labels, train_mask, val_mask, test_mask, evaluator, edge_weight=edge_weights)

    if epoch % log_steps == 0:
        train_acc, valid_acc, test_acc = result
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 100, Loss: 1.3268, Train: 65.77%, Valid: 66.36% Test: 65.61%
Epoch: 200, Loss: 1.1211, Train: 70.01%, Valid: 69.28% Test: 68.27%
Epoch: 300, Loss: 1.0365, Train: 71.66%, Valid: 70.32% Test: 69.46%
Epoch: 400, Loss: 0.9862, Train: 72.85%, Valid: 70.87% Test: 69.94%
Epoch: 500, Loss: 0.9477, Train: 73.67%, Valid: 71.22% Test: 70.30%
Epoch: 600, Loss: 0.9254, Train: 74.37%, Valid: 71.50% Test: 70.52%
Epoch: 700, Loss: 0.8974, Train: 75.03%, Valid: 71.85% Test: 70.88%
Epoch: 800, Loss: 0.8785, Train: 75.65%, Valid: 72.10% Test: 71.17%
Epoch: 900, Loss: 0.8579, Train: 76.14%, Valid: 72.08% Test: 71.12%
Epoch: 1000, Loss: 0.8428, Train: 76.58%, Valid: 72.36% Test: 71.57%
Epoch: 1100, Loss: 0.8278, Train: 77.03%, Valid: 72.41% Test: 71.61%
Epoch: 1200, Loss: 0.8154, Train: 77.46%, Valid: 72.56% Test: 71.51%
Epoch: 1300, Loss: 0.8021, Train: 77.85%, Valid: 72.64% Test: 71.38%
Epoch: 1400, Loss: 0.7903, Train: 78.29%, Valid: 72.89% Test: 71.74%
Epoch: 1500, Loss: 0.7800, Train: 78.59%, V